In [30]:
import json
import os

import pandas as pd

from config import DATA_PATH, VISUALIZATIONS_PATH
from utilities import SPACY_MODEL, preprocess_text

In [53]:
session_speech = pd.read_csv(
    os.path.join(DATA_PATH, "session_speech_speaker_info.csv"),
    converters={"speech": eval},
    )
session_speech.head(2)

,name,vote,senator,province,party,party_family,speaker,speech
0,Ana Claudia Almirón,positivo,ALMIRÓN ANA CLAUDIA,CORRIENTES,ALIANZA FRENTE PARA LA VICTORIA,Frente para la victoria,Almirón,"[Hace dos años, tuvimos una sesión histórica c..."
1,Roberto Gustavo Basualdo,negativo,BASUALDO ROBERTO GUSTAVO,SAN JUAN,ALIANZA CAMBIEMOS SAN JUAN,Juntos por el cambio,Basualdo,"[Gracias, señor presidente. Hoy es un día en e..."


In [55]:
session_speech = (
    session_speech
    .explode("speech")
    .dropna(subset=["speech"], ignore_index=True)
)
session_speech.head(2)

,name,vote,senator,province,party,party_family,speaker,speech
0,Ana Claudia Almirón,positivo,ALMIRÓN ANA CLAUDIA,CORRIENTES,ALIANZA FRENTE PARA LA VICTORIA,Frente para la victoria,Almirón,"Hace dos años, tuvimos una sesión histórica cu..."
1,Roberto Gustavo Basualdo,negativo,BASUALDO ROBERTO GUSTAVO,SAN JUAN,ALIANZA CAMBIEMOS SAN JUAN,Juntos por el cambio,Basualdo,"Gracias, señor presidente. Hoy es un día en el..."


In [56]:
out_file = os.path.join(DATA_PATH, "words2lemmas.json")

with open(out_file, "r") as f:
    words2lemmas = json.load(f)

words2lemmas["es"]

{'VERB': {'lemma': 'ser', 'count': 1247}}

In [63]:
def select_lemma_and_pos(text: str) -> tuple[str, str]:
    lemmatized_text, pos_text = "", ""
    tokens = SPACY_MODEL(text)
    for t in tokens:
        token = preprocess_text(t.text)
        lemmas = words2lemmas.get(token)
        if lemmas is None:
            continue
        if len(lemmas) == 1:
            lemma_value = list(lemmas.values())[0]
            pos = list(lemmas.keys())[0]
        else:
            lemma_value = lemmas.get(t.pos_)
            if lemma_value is None:
                lemma_count = 0
                for key, value in lemmas.items():
                    if value["count"] > lemma_count:
                        lemma_count = value["count"]
                        pos = key
                        lemma_value = value
        lemma = lemma_value.get("lemma")
        if (lemma is None) or (pos is None):
            raise Exception(
                f"Error while processing token {token} ({t.pos}), lemmas found {lemmas}"
            )
        lemmatized_text += f"{lemma} "
        pos_text += f"{pos} "
    return lemmatized_text.strip(), pos_text.strip()

In [66]:
session_speech["speech_lemmas"], session_speech["speech_pos"] = (
    zip(*session_speech.speech.apply(select_lemma_and_pos))
)
session_speech[["speech", "speech_lemmas", "speech_pos"]].head(2)

,speech,speech_lemmas,speech_pos
0,"Hace dos años, tuvimos una sesión histórica cu...",hacer año tener una sesión histórica cuando de...,VERB NOUN VERB DET NOUN ADJ SCONJ VERB ADP ADJ...
1,"Gracias, señor presidente. Hoy es un día en el...",gracia señor presidente hoy ser un día en el q...,NOUN NOUN NOUN ADV VERB DET NOUN ADP DET SCONJ...
